### Build a tile export service mixing precalculated tiles with live tiles from Earth Engine

This notebook shows an example of how to mix pre-calculated tiles from Earth Engine (hosted on GCS) with live-calculated tiles from Earth Engine (past zoom level 12).

Based on code snippets drawn from [here](https://github.com/wri/gfw-api/blob/master/gfw/gee_tiles.py).

Note: the tokens for the live Earth Engine tiles expire every 3 days, and so this code will need to be re-run with an authorised EE account to generate valid (live) tiles.

In [1]:
import os
import ee
import json

ee.Initialize()

In [2]:
def tile_url(image, viz_params=None):
    """Create a target url for tiles for an image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (base_url + '/map/' + d['mapid'] + '/{z}/{x}/{y}?token=' + d['token'])
    return url

### EE pansharpened image

Below calculates a pan-sharpended image from Landsat, and creates a tile url with temporary authentitication.

In [3]:
collection = ee.ImageCollection('LANDSAT/LC8_L1T').filterDate('2016-01-01T00:00','2017-01-01T00:00')

composite = ee.Algorithms.SimpleLandsatComposite(collection=collection, percentile=50,
                                                 maxDepth=80, cloudScoreRange=1, asFloat=True)

hsv2 = composite.select(['B4', 'B3', 'B2']).rgbToHsv()

sharpened2 = ee.Image.cat([hsv2.select('hue'), hsv2.select('saturation'),
                           composite.select('B8')]).hsvToRgb().visualize(gain=1000, gamma= [1.15, 1.4, 1.15])


ee_tiles = tile_url(sharpened2)
ee_tiles

'https://earthengine.googleapis.com/map/76b85de109f4c459eea93a4029ccab67/{z}/{x}/{y}?token=6bd268d164561ff1471cefe2aa73c38e'

In [4]:
pre_calculated_tileset="https://storage.googleapis.com/landsat-cache/2016/{z}/{x}/{y}.png"

In [5]:
import folium

In [6]:
map = folium.Map(location=[28.29, -16.6], zoom_start=4, tiles='Mapbox Bright' )

In [7]:
map.add_tile_layer(tiles=pre_calculated_tileset, max_zoom=12, min_zoom=4, attr='Earth Engine tiles by Vizzuality')
map.add_tile_layer(tiles=ee_tiles, max_zoom=20, min_zoom=13, attr="Live EE tiles")

In [8]:
map